In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import re
from multiprocessing import Pool
from functools import partial

In [ ]:
pip install selenium

In [30]:
class IndeedItemParser:
    def __init__(self):
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        #self.utilities = utilities();
        
    def _get_title(self, driver):
        try:
            title = driver.find_element_by_xpath("//*[@class='jobsearch-DesktopStickyContainer']//h3")
        except Exception as e:
            print(e)
        return title.text
    
    def _get_name(self, driver):
        
        try:    
            name = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[1]")
            return name.text
        except Exception as e:
            print(e)
        
    
    def _get_address(self,driver):
         try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[3]")
            if address.text == "-":
                address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[4]")
            return address.text
         except Exception as e:
            address = driver.find_element_by_xpath("//span[@class='jobsearch-JobMetadataHeader-iconLabel'][1]")
            return address.text
    
    def _get_salaire(self,driver, description):
        return np.nan # à compléter dans la partie pre-processing
            
    
    def _get_description(self,driver):
        try:
            #jobDescriptionText
            e_description = driver.find_element_by_id("jobDescriptionText")
            return e_description.text
        except Exception as e:
            print(e)
    
    def _get_date(self,driver,url,name):
       # print(url)
        try:
            date_str = driver.find_element_by_xpath("//*[@class='jobsearch-JobMetadataFooter']")
            date_str_full = date_str.text
            date_tbl = date_str_full.split(" ")
            count_str = date_tbl[4]
            label = date_tbl[5]
            
            if name in date_str_full:
                date_str_full = date_str_full.replace(name, "")
                date_tbl = date_str_full.split(" ")
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            if count_str == "a" :
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            #print("date_str", date_tbl)
            date = datetime.now()
            
            if count_str == "30+":
                return date - timedelta(days=30)
            
            count = int(count_str)
            if "jour" in label:
                date = date - timedelta(days=count)
            elif "heur" in label:
                date = date - timedelta(hours=count)
            return date;
        except Exception as e:
            print(e)
    
    
    def parse(self,url):
        driver = webdriver.Chrome(self.driverPath)
        driver.get(url)
        driver.maximize_window()
        
        source = driver.page_source
        title = self._get_title(driver)
        name = self._get_name(driver)
        address = self._get_address(driver)
        date = self._get_date(driver, url,name)
        description = self._get_description(driver)
        salaire = self._get_salaire(driver,description)
        
        driver.quit()
        
        return title, name, address, date, salaire, description, source

In [31]:
class KeyWordsProvider:
    
    def get_langages(self):
        return ['python', 'r','vba', 'mysql','excel','asp.net','nosql','sql','linux','mongodb',
            'mariadb','java','javascript','php','html','css','sas','c#','ruby','swift','objective-c',
            'vb.net','kotlin','scala','bash','powershell','shell','front end','back end','soap']
    
    def get_tools(self):
        return ['tableau', 'powerbi','symfony', 'jquery','angular','react','react native','node js','git','github',
            'visual studio','django','flask','api rest','laravel', 'hololens', 'docker', 'jira', 'scrum', 'kanban', 
               'azure','aws', 'teamcity', 'jenkins']
    
    

In [34]:
from datetime import datetime, timedelta
import os.path
import os
class IndeedPaser:
    def __init__(self):
        self.website = "https://www.indeed.fr"
        self.driverPath = "C:\\Users\\Junior\\Documents\\selenium\\driver\\chromedriver.exe"
        
        self.dataset = pd.read_csv("indeed.csv")
        
        self.jobs = ["développeur", "data scientist", "data analyst", "business intelligence"]
        self.locations = ["Paris","Lyon", "Toulouse", "Nantes", "Bordeaux"]
        #self.utilities =  utilities()
        self.indeed_item_parser = IndeedItemParser()
        self.parser_state = IndeedParserStateHandler()
        self.keyWordsProvider = KeyWordsProvider()
    
    def _get_pages_counts(self,driver):
        searchCountPages_elt = driver.find_element_by_id("searchCountPages")
        searchCountPages = searchCountPages_elt.text.split()
        if len(searchCountPages) == 6:
            searchCountPages = int("{0}{1}".format(searchCountPages[3],searchCountPages[4])) 
        else :
            searchCountPages = searchCountPages[3]  
        
        return (int(searchCountPages) // 18)
    
    def _get_subs_collections(self,items, nbr=5):
        result = []
        sub = []
        for index, item in enumerate(items):
            sub.append(item)
            if (index > 0) & (index % nbr) == 0:
                result.append(sub)
                sub = []
        return result
    
    def _local_parse_page(self, item_link, localtion, index_i, dataset_len):
        try:
            if (self.dataset["URL"].isin([item_link]).any()):
                print("aready parsed, skip", item_link)
            else:
                title, name, address, date,salaire, description, source = self.indeed_item_parser.parse(item_link)
                if (self.dataset["Titre"].isin([title]).any() 
                    & self.dataset["Nom entreprise"].isin([name]).any() 
                    & self.dataset["Adresse"].isin([address]).any()
                    & self.dataset["description"].isin([description]).any()):
                    print("doublon, skip.")
                    return
                
                cols = [item_link, title, name, address, date, salaire, description,localtion ]
                self.dataset.loc[dataset_len + index_i] = cols
                source_file_name = "source_file.{0}.html".format(dataset_len + index_i)
                if not os.path.exists(source_file_name):
                    file = open("pages/{0}".format(source_file_name), "w")
                    source = item_link + "----------------" + source
                    file.write(source)
                    file.close()
                print("len(dataset)", len(self.dataset))
        except Exception as e:
            print(e)
    
    
    def parse(self):
        browser = webdriver.Chrome(self.driverPath)
        browser.get(self.website)
        browser.maximize_window()
        
        for job in self.jobs:
            jobs_filter_list = [job]
            
            if job == "développeur":
                all_competences = self.keyWordsProvider.get_langages() + self.keyWordsProvider.get_tools()
                jobs_filter_list = ["{0} {1}".format(job, item) for item in all_competences]
            
            for job_key_word in jobs_filter_list:
                for location in self.locations:

                    query = "https://www.indeed.fr/jobs?q={0}&l={1}".format(job_key_word, location)
                    browser.get(query)

                    pages_count = self._get_pages_counts(browser)

                    for page_index in range(1, pages_count):
                        full_query = "{0}&start={1}".format(query,page_index)

                        browser.get(full_query)

                        items = browser.find_elements_by_xpath("//*[contains(@class,'clickcard')]//*[contains(@class,'jobtitle')]")
                        items = [item.get_attribute("href") for item in  items]
                        dataset_len = len(self.dataset)
                        
                        #collections = self._get_subs_collections(items)
                        for index_i, link in enumerate(items):
                            self._local_parse_page(link, location, index_i,dataset_len)
                        
                        self.dataset.to_csv("indeed.csv",index=False)
                        #print("indeed.csv saved")
                        self.dataset = pd.read_csv("indeed.csv")
                
              

In [35]:
test = IndeedPaser()
test.parse()

doublon, skip.
doublon, skip.
len(dataset) 20
doublon, skip.
doublon, skip.
len(dataset) 21
aready parsed, skip https://www.indeed.fr/rc/clk?jk=7de11d138b3a2b85&fccid=0bc9d524dbf6f889&vjs=3
aready parsed, skip https://www.indeed.fr/company/Flowka/jobs/D%C3%A9veloppeur-Fullstack-Junior-0f4aa189d14603a5?fccid=0a8ff107c11b8f22&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=96c9eb4c4a5cca53&fccid=1c939006b684b25c&vjs=3
aready parsed, skip https://www.indeed.fr/company/LEC-Conseil/jobs/D%C3%A9veloppeur-Fullstack-F-H-f5b68ff17d9cd4f4?fccid=3d3e7ac112fae637&vjs=3
len(dataset) 22
aready parsed, skip https://www.indeed.fr/company/OCTOPUS-IT/jobs/D%C3%A9veloppeur-Python-Junior-Dipl%C3%B4m%C3%A9-Logiciel-Open-Source-c40fbad4cbf04651?fccid=7358eb8948faec3a&vjs=3
aready parsed, skip https://www.indeed.fr/company/Data-Impact-Analytics/jobs/D%C3%A9veloppeur-Python-Junior-1a815b2313a4f6fc?fccid=f117fa20a31c19b8&vjs=3
len(dataset) 23
len(dataset) 24
len(dataset) 25
doublon, skip.
doublon, sk

KeyboardInterrupt: 

In [ ]:
import numpy as np 
class IndedRegexParser:
    
    def __init__(self, descriptif_text ):
        self.descriptif_text = descriptif_text
    
    def _get_items(self,pattern):   
        result_re = re.findall(pattern,self.descriptif_text)
        result = []
        for item_l_1 in result_re:
            for item_l_2 in item_l_1:
                if item_l_2 != "":
                    result.append(item_l_2)
        return result
    
    def _get_niveau_experience(self):
        #Niveau d'expérience : regex true
        result = self._get_items('([(e|E)\w+]xpérience?:?\s\d{1,15} ans?)|([(e|E)\w+]xpérience?\s?de\s?\d{1,15} ans?)|(\d{1,15} ans d\'[e|E]xpérience[s]?)')
        if len(result) == 0:
            outer = re.compile(r'Expérience:\s*.+:( (\w) (an|ans) \(((R|r)equis|(s|S)ouhaité)\))')
            m = outer.search(self.descriptif_text)
            if m is not None:
                return m.group(1)
            #return []
    
    def _get_salaires(self):
        #Salaire ajusté en fonction du coût de la vie dans les différentes villes : regex true
        #return self._get_items('(\d{2,3}\s?(k|K)?)|(\d{2,3}\s?\d{3,5}\s?(k|K)?)|([(s|S)]alaire?[\s]?:?[\s]?\d{2,6}k?)|([(r|R)]émunération?[\s]?:?[\s]?\d{2,6}k?)|([(g|G)]ratification?[\s]?:?[\s]?\d{2,6}k?)') 
        #[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/an
        outer = re.compile(r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|(.*)?[\s+]?par?[\s+]?mois')
        m = outer.search(self.descriptif_text)
        if m is not None:
            return m.group(1)
        return []
            
        
    
    def get_langages_prog(self):
        #Langages de programmation requis (R, Python, Excel (VBA), SQL, C++, Java, SAS...): regex true
        return self._get_items('(R+[^\w])|([(p|P)\w+]ython?)|(Excel?)|(VBA?)|(C\+\+?)||(C?)|(Asp.net?)|(SQL?)|(NoSQL?)|(Linux?)|(MySQL?)|(MongoDB?)|(DBMaria?)|(SQL Server?)||(Java?)|(JavaScript?)|(PHP?\s?\d{1,}?)|(HTML?\s?\d{1,}?)|(CSS?\s?\d{1,}?)|(SAS?)|(C#?)|(Ruby?)|(Swift?)|(Objective-C?)|(VB.NET?)|(Kotlin?)|(Scala?)|(Bash?)|(PowerShell?)|(Shell?)|(Front End?)|(Back End?)')
    
    def get_outils_re(self):
        #Outils requis (Tableau, PowerBI...) : regex true
        return self._get_items('([(t|T)\w+]ableau?)|([(p|P)\w+]ower[(b|B)\w+][(i|I)\w+]?)|([(s|S)\w+]ymfony?\s?\d{1,}?)|([(j|J)\w+][(q|Q)\w+]uery?\s?\d{1,}?)|([(a|A)\w+]ngular?[\s?\d{1,}]?)|([(r|R)\w+]eact [(j|J)\w+][(s|S)\w+]?)|(React Native?)|(Node JS?)|([(g|G)\w+]it?)|(Visual Studio?)|(Visual Studio Code?)|(Django?)|(Flask?)|(API REST?)|(http?s?)|(HTTP?S?)')
        
        
    def get_niv_etudes_re(self):
        #Niveau d'études requis : regex true
        return self._get_items('([(f|F)\w+]ormation?\s?[(b|B)\w+]ac\s\+\s[1-8])|([(b|B)\w+]ac\s\+\s[1-8])')
        
    def get_type_cursus_re(self):
        #Type de cursus (école ingénieur, master, autodidacte...) : regex true
        return self._get_items('([(é|E)\w+]cole [(i|I)\w+]ngénieur?)|([(m|M)\w+]aster?\s?\w{3,25})|([(a|A)\w+]utodidacte?)|([(g|G)\w+]rande[s]? [(é|E)\w+]cole[s]?)|([(é|E)\w+]cole[s]? de [(c|C)\w+]ommerce[s]?)|([(i|I)\w+]ngénieur [(i|I)\w+]nformatique?)')
        
    def get_type_contrat_re(self):
        #Type de contrat : regex true
        return self._get_items('[(c|C)\w+]ontrat?:?\s\w{3,25}')
    
    def get_categorie_developpeur_re(self):
        #Grande catégorie : développeur (developeur web, dev mobile,dev) / data regex true
        return self._get_items('([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ull? [(s|S)\w+]tack)|([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ront[(\s|\-)][(e|E)\w+]nd?)|(développeu[(r|s)]e? Back[(\s|\-)][(e|E)\w+]nd?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(m|M)\w+]obile?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciel?)|([(s|S)\w+]oftware [(i|I)\w+]ngenieer?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciels?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(C|c)\w+]\+\+?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(c|C)\w+]#?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(j|J)\w+]ava?)')
    
    def get_categorie_data_re(self):
        #Grande catégorie : data regex true
        return self._get_items('([(d|D)\w+]ata [(a|A)\w+]nalyst?)|([(d|D)\w+]ata [(s|S)\w+]cientist?)|([(i|I)\w+]ngénieu[(r|s)]e? [(m|M)\w+]achine [(i|I)\w+]earning?)|([(c|C)\w+]onsultant [(b|B)\w+]usiness [(i|I)\w+]ntelligence)|([(c|C)\w+]onsultant [(d|D)\w+]ata)|([(c|C)\w+]onsultant [(b|B)\w+]ig [(d|D)\w+]ata)|([(c|C)\w+]onsultant BI)')
        

In [ ]:
dataset = pd.read_csv("indeed.csv")
dataset
#print(len(dataset))
#for index, row in dataset.iterrows():
    #print(row["URL"])
    #indedRegexParser = IndedRegexParser(row["description"])
    #print(indedRegexParser._get_niveau_experience())
    #print(indedRegexParser._get_salaires())
    #print(indedRegexParser.get_outils_re())
    #print(indedRegexParser.get_niv_etudes_re())
    #print(indedRegexParser.get_type_cursus_re())
    #print(indedRegexParser.get_type_contrat_re())
    #print(indedRegexParser.get_categorie_developpeur_re())
    #print(indedRegexParser.get_categorie_data_re())


In [ ]:
dataset = pd.read_csv("indeed.csv")
type(dataset["salaire"][18])

In [ ]:
import os
from orderedset import OrderedSet
from bs4 import BeautifulSoup
class LocalIndeedPaser:
    def __init__(self):
        self.pages_path = r'C:\Users\Junior\Documents\Projects_Simplon\Projet_ML_gr3-master\scrapping\pages'
        self.dataset = pd.read_csv("indeed.csv")
        self.salary_pattern = "[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|((.*)?[\s+]?par?[\s+]?(an|ans|mois|jour|heure))"
        self.keyWordsProvider = KeyWordsProvider()
    
    def _get_salary(self,select_result):
        salary = ""
        for item in select_result:
            if "€" in item.text:
                outer_salary = re.compile(self.salary_pattern)
                m_salary = outer_salary.search(item.text)
                if m_salary is not None:
                    salary = m_salary.group(0)
                    break
        return salary
    
    def save_file(self):
        self.dataset.to_csv("indeed.csv",index=False)
    
    def _get_binnary_list_data(self, input_list):
        data = []
        for i in range(len(self.dataset)):
            inside_data = []
            for ele in input_list:
                pattern = re.compile(r"[\s/\(\),]"+ele+r"[\s/\(\),]")
                value = pattern.search(self.dataset['description'][i].lower().replace('\n',' ').replace('\r',' '))
                if value:
                    inside_data.append(1)
                else:
                    inside_data.append(0)
            data.append(inside_data)
        
        return data
    
    def _set_quantitative_features(self, pattern, col_indice,label_col,func_callback = None):
        result = []
        for index, row in self.dataset.iterrows():
            re_pattern = re.compile(pattern)
            value = re_pattern.search(row['description'].lower().replace('\n',' ').replace('\r',' '))
            if value:
                if func_callback is not None:
                    result.append(func_callback(value.group(0)))
                else:
                    result.append(value.group(0))
            else:
                result.append(None)
        
        if (label_col not in self.dataset.columns):
            self.dataset.insert(col_indice, label_col,result,True)
        else:
            self.dataset[label_col] = pd.DataFrame(result)
        
        dummies = self.dataset[label_col].str.get_dummies() 
        
        self._fusion_with_dataset(dummies)
        
    
    def parse_education_level(self):
        reg_pattern = "([(b|B)\w+]ac\s*\+\s*[1-8])|ingénieur|master\s*(1|2)|(D|d)iplôme\s*supérieur"
        self._set_quantitative_features(reg_pattern,7,"niveau_etude", self._education_level_callback)
    
    def _education_level_callback(self, value):
        bac_pattern = "bac\s*\+\s*[1-8]"
        result = re.compile(bac_pattern).search(value)
        response = ""
        if result:
            response = re.findall('(\d+)',value)
            if response is not None:
                return "bac + " + response[0] 
        
        master_pattern = "master\s*(1|2)"
        result = re.compile(master_pattern).search(value)
        if result:
            response = re.findall('(\d+)',value)
            if response is not None:
                return "master " + response[0] 
            
        return value
    
    def set_type_de_cursus(self):
        #j'ai desactivé le pattern "|([(m|M)\w+]aster?\s?\w{3,25})" master car ça renvoie "master dans" ou "master data"
        reg_pattern = '([(é|E)\w+]cole [(i|I)\w+]ngénieur?)|([(a|A)\w+]utodidacte?)|([(g|G)\w+]rande[s]? [(é|E)\w+]cole[s]?)|([(é|E)\w+]cole[s]? de [(c|C)\w+]ommerce[s]?)|([(i|I)\w+]ngénieur [(i|I)\w+]nformatique?)'
        self._set_quantitative_features(reg_pattern,8,"type_de_cursus", self._type_cursus_callback)
    
    def _type_cursus_callback(self, value):
        ge_pattern = "grandes?\s*(é|e)coles?"
        result = re.compile(ge_pattern).search(value)
        if result:
            return "grande école"
        
        ec_pattern = "([(é|E)\w+]cole[s]? de [(c|C)\w+]ommerce[s]?)"
        result = re.compile(ec_pattern).search(value)
        if result:
            return "école de commerce"
        
        return value
        
    def set_type_de_contrat(self):
        #j'ai desactivé le pattern "[(c|C)\w+]ontrat?:?\s\w{3,25}|"  car ça renovie "contrat logue", "contrat avec", etc
        reg_pattern = '(cdi|cdd|stage|alternance|alternant|cdic|freelance)|3\s*mois\s*renouvelable\s*'
        self._set_quantitative_features(reg_pattern,9,"type_de_contrat", self._type_de_contrat_callback)
    
    def _type_de_contrat_callback(self,value):
        bac_pattern = "alternance|alternant"
        result = re.compile(bac_pattern).search(value)
        if result:
            return "alternance"
        return value
    
    def set_grande_categorie(self):
        reg_pattern = 'développeur?\s*(web|mobile|data|front\s*end|back\s*end|desktop|full stack\s*(developer))'
        self._set_quantitative_features(reg_pattern,10,"grande_categorie",self._grande_categorie_callback)
        
    def _grande_categorie_callback(self, value):
        bac_pattern = "front\s*end|back\s*end"
        result = re.compile(bac_pattern).search(value)
        if result:
            if 'front' in value:
                return "front-end"
            else:
                return "back-end"
            
        return value
    
    def _fusion_with_dataset(self, df):
        diff_cols = list(OrderedSet(self.dataset.columns) - OrderedSet(df.columns))
        self.dataset = self.dataset[diff_cols]
        self.dataset = pd.concat([self.dataset,df], axis=1)
    
    def parse_langage(self):
        languages = self.keyWordsProvider.get_langages()
        
        data = self._get_binnary_list_data(languages)
        
        language_dict = pd.DataFrame(data, columns=languages)
        self.language_df = pd.DataFrame.from_dict(language_dict)

        self._fusion_with_dataset(self.language_df)
    
    def parse_tools(self):
        tools = self.keyWordsProvider.get_tools()
        
        data = self._get_binnary_list_data(tools)
        tools_dict = pd.DataFrame(data, columns=tools)
        self.tools_df = pd.DataFrame.from_dict(tools_dict)
        self._fusion_with_dataset(self.tools_df)
    
    def set_salary_man(self):
        salaire_moyen = []
        for i in range(len(self.dataset)):
            try:
                salaire_liste = re.findall('(\d+),?',normalize('NFKD',df['salaire'][i]).replace(' ',''))
                mois = re.search('mois',df['salaire'][i])
                if mois:
                    if len(salaire_liste) > 1:
                        moy = 12 * (int(salaire_liste[0]) + int(salaire_liste[1])) / 2
                        salaire_moyen.append(moy)
                    else:
                        salaire_moyen.append(int(salaire_liste[0]) * 12)
                else:
                    if len(salaire_liste) > 1:
                        moy = (int(salaire_liste[0]) + int(salaire_liste[1])) / 2
                        if moy < 100:
                            moy *= 1000
                        salaire_moyen.append(moy)
                    else:
                        if int(salaire_liste[0]) < 100:
                            salaire_moyen.append(int(salaire_liste[0]) * 1000)
                        else:
                            salaire_moyen.append(int(salaire_liste[0]))

            except:
                salaire_moyen.append(None)
                continue
        
        label_col = "salaire_moyen"
        
        if (label_col not in self.dataset.columns):
            self.dataset.insert(6, label_col,salaire_moyen,True)
        else:
            self.dataset[label_col] = pd.DataFrame(salaire_moyen)
            self.dataset[label_col]
    
    def parse_salary_from_local_files(self):
        index = 0
        for root, dirs, files in os.walk(os.path.abspath(self.pages_path)):
            for file in files:
                local_file = os.path.join(root, file)
                
                with open(local_file,"r" ) as file:
                    html = file.read() 
                    outer = re.compile(r'.*\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-')
                    m = outer.search(html)
                    if m is not None:
                        link = m.group(0).replace("----------------","")
                        dataset_index = self.dataset[self.dataset["URL"] == link].index.values.astype(int)
                        salary = self.dataset["salaire"][dataset_index]
                        if (len(salary.values) == 0):
                            continue 

                        if (salary.values[0] != "-"):
                            continue

                        html = html.replace(m.group(0),"")
                        soup = BeautifulSoup(html, "html.parser")
                        result = soup.select(".jobsearch-JobMetadataHeader-item")
                        if len(result) > 0:
                            self.dataset["salaire"][dataset_index] = self._get_salary(result)
                            continue

                        result = soup.select(".jobsearch-JobMetadataHeader-itemWithIcon .jobsearch-JobMetadataHeader-iconLabel")
                        if len(result) > 0:
                            self.dataset["salaire"][dataset_index] = self._get_salary(result)
                            continue

                        result = soup.select("#jobDescriptionText")
                        if len(result) > 0:
                            index = index + 1 
                            outer_salary = re.compile(self.salary_pattern)
                            m_salary = outer_salary.search(result[0].text)
                            if m_salary is not None:
                                salary = m_salary.group(0)
                                print(salary)
                                continue

                        salary = np.nan #set nan if we cannot get value
                    
                    

In [ ]:
localIndeedPaser = LocalIndeedPaser()
#localIndeedPaser.dataset["description"].dropna(how='all').isna().sum()
localIndeedPaser.dataset = localIndeedPaser.dataset.dropna(subset=['description'])
localIndeedPaser.dataset["description"].isna().sum()

In [ ]:
localIndeedPaser.parse_education_level()
localIndeedPaser.set_type_de_cursus()
localIndeedPaser.set_type_de_contrat()
localIndeedPaser.set_grande_categorie()
localIndeedPaser.parse_langage()
localIndeedPaser.parse_tools()

In [ ]:
localIndeedPaser.parse_salary_from_local_files()
localIndeedPaser.set_salary_man()

In [ ]:
localIndeedPaser.dataset["Date de publication"].isna().sum()
#localIndeedPaser.dataset[localIndeedPaser.dataset["Date de publication"] == "NaN"]

In [ ]:
for i in localIndeedPaser.dataset.columns:
    print(i)

In [ ]:
pip install orderedset

In [ ]:
localIndeedPaser.save_file()

In [ ]:
xx

In [ ]:
xx = pd.read_csv("indeed.csv")
xx

In [ ]:
xx